In [1]:
# PART 1
# empty lists

link = []
name = []
price = []
rating = []
review = []

In [2]:
def extract_info(bags) :
    for bag in bags :

        # Link
        try :
            a_tags = bag.find_elements_by_tag_name('a')
            product_link = ''
            for tag in a_tags :
                if 'bestsellers' not in str(tag.get_attribute('href')) :
                    product_link = (tag.get_attribute('href'))
                    break
            link.append(product_link)
        except :
            link.append("NA")

        # Name
        try :
            curr_name = bag.find_element_by_class_name('a-size-medium').text
            name.append(curr_name)
        except :
            name.append("NA")

        # Price
        try :
            curr_price = bag.find_element_by_class_name('a-price-whole').text
            price.append(curr_price)
        except :
            price.append("NA")

        # Rating
        try :
            curr_rating = bag.find_elements_by_class_name('a-size-base')[0].text
            rating.append(curr_rating)
        except :
            rating.append("NA")

        # Review
        try :
            curr_review = bag.find_elements_by_class_name('a-size-base')[1].text
            review.append(curr_review)
        except :
            review.append("NA")

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
PATH = "C:\selenium_driver\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get('https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1')

In [4]:
bags = driver.find_elements_by_xpath('//div[@class="sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16"]')
extract_info(bags)

# looping over 30 pages
for i in range(30) :
    try :
        # Find the "Next" button and click it
        next_button = driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
        next_button.click()
        # 10 sec sleep
        time.sleep(10)
        # extends the list informations
        extract_info(driver.find_elements_by_xpath('//div[@class="sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16"]')) # all bags
        # Wait for the next page of search results to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='s-result-list s-search-results sg-row']")))

    except Exception as e :
        print(e)
    print(len(link))

Message: 

32
Message: 

48
Message: element click intercepted: Element <a href="/s?k=bags&amp;page=4&amp;crid=2M096C61O4MLT&amp;qid=1679497953&amp;sprefix=ba%2Caps%2C283&amp;ref=sr_pg_3" aria-label="Go to next page, page 4" class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator">...</a> is not clickable at point (574, 257). Other element would receive the click: <div class="rhf-sign-in-button">...</div>
  (Session info: chrome=111.0.5563.65)

48
Message: 

64
Message: 

80
Message: 

96
Message: 

112
Message: 

128
Message: 

144
Message: 

160
Message: 

176
Message: 

192
Message: 

208
Message: 

224
Message: 

240
Message: 

256
Message: 

272
Message: 

288
Message: 

304
Message: 

306
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']"}
  (Session info: chrome=111.0.5563.65)

306
Message: no such element: Unable to locate el

In [5]:
import pandas as pd

data_bags = pd.DataFrame({
    "link" : link,
    "name" : name,
    "price" : price,
    "rating" : rating,
    "review" : review
})

,link,name,price,rating,review
0,https://www.amazon.in/American-Tourister-AMT-S...,American Tourister 32 Ltrs Black Casual Backpa...,"1,199",4.0,"(52,349)"
1,https://www.amazon.in/Skybags-Brat-Black-Casua...,Skybags Brat Black 46 Cms Casual Backpack,669,4.1,"(3,791)"
2,https://www.amazon.in/Wesley-Milestone-Waterpr...,Wesley Milestone 2.0 Casual Waterproof Laptop ...,565,4.3,"(9,839)"
3,https://www.amazon.in/ADISA-Laptop-Backpack-Of...,ADISA 15.6 inch Laptop Backpack Office Bag Col...,499,3.9,(573)
4,https://www.amazon.in/Amazon-Brand-Presto-Oxo-...,Amazon Brand - Presto! Oxo-Biodegradable Garba...,339,4.3,"(30,163)"


In [24]:
# PART 2

def extract_product_details(product_url) :
    try :
        driver.get(product_url)
        product_details = (driver.find_element_by_id('detailBullets_feature_div')).find_elements_by_tag_name('li')
        details_dict = {}

        for pd in product_details :
            details_list_item = (pd.find_element_by_class_name('a-list-item')).find_elements_by_tag_name('span')
            key = details_list_item[0].text
            value = details_list_item[1].text
            if 'Manufacturer' in key :
                details_dict['Manufacturer'] = value
            # asin
            elif 'ASIN' in key :
                details_dict['ASIN'] = value

        # product description
        try :
            description_element = driver.find_element_by_id("productDescription_feature_div")
            description = (description_element.find_element_by_tag_name('p')).text
        except :
            description = "NA"

        # manufacturer
        try :
            manufacturer = details_dict['Manufacturer']
        except :
            manufacturer = "NA"

        # asin
        try :
            asin = details_dict['ASIN']
        except :
            asin = "NA"


        return manufacturer, asin, description
    except :
        return "NA", "NA", "NA"

In [25]:
manufacturer_list = []
asin_list = []
desc_list = []

for it in data_bags['link'] :
    m, a, d = extract_product_details(it)
    manufacturer_list.append(m)
    asin_list.append(a)
    desc_list.append(d)

In [26]:
data_bags["manufacturer"] = manufacturer_list
data_bags["ASIN"] = asin_list
data_bags["description"] = desc_list

In [27]:
data_bags.head()

,link,name,price,rating,review,manufacturer,ASIN,description
0,https://www.amazon.in/American-Tourister-AMT-S...,American Tourister 32 Ltrs Black Casual Backpa...,"1,199",4.0,"(52,349)",NA,NA,NA
1,https://www.amazon.in/Skybags-Brat-Black-Casua...,Skybags Brat Black 46 Cms Casual Backpack,669,4.1,"(3,791)",VIP Industries Ltd,B08Z1HHHTD,Ideal for a college student who does not carry...
2,https://www.amazon.in/Wesley-Milestone-Waterpr...,Wesley Milestone 2.0 Casual Waterproof Laptop ...,565,4.3,"(9,839)",NA,NA,NA
3,https://www.amazon.in/ADISA-Laptop-Backpack-Of...,ADISA 15.6 inch Laptop Backpack Office Bag Col...,499,3.9,(573),"ADISA, ADISA INTERNATIONAL, 42/34, Upper Groun...",B09TPX22NF,NA
4,https://www.amazon.in/Amazon-Brand-Presto-Oxo-...,Amazon Brand - Presto! Oxo-Biodegradable Garba...,339,4.3,"(30,163)",NA,NA,NA


In [30]:
data_bags.to_csv('final amazon bag data.csv', index=False)

In [ ]:
# THANK YOU